# Setup

In [1]:
import contextily as cx
import figure_utilities
import statsmodels.api as sm
import constants
from stats_utilities import produce_summary_statistics, select_controls, test_balance
import geopandas as gpd
import matplotlib.pyplot as plt
from panel_utilities import get_value_variable_names, prepare_df_for_DiD
import numpy as np
plt.rcParams['savefig.dpi'] = 300
import statsmodels.api as sm
import os
import pandas as pd

/Users/ashanmu1/Documents/GitHub/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/d2/xc40wfzn765fbyjgk7fz021r0000gn/T/ipykernel_57322/1283802068.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEO

In [2]:
# Store paths.
INPUT_DATA_PANEL = "../data/03_cleaned/crime_analysis_monthly.csv"
INPUT_DATA_TRACTS = "../data/02_intermediate/tracts.csv"
INPUT_DATA_NEIGHBORHOOD_CRIME_COUNTS = "../data/03_cleaned/neighborhood_crime_counts.csv"
INPUT_DATA_NEIGHBORHOODS = "../data/01_raw/2015-2019_neighborhood_tables_2021.12.21.xlsm"
INPUT_DATA_BOSTON_TRACTS_SHAPEFILE = "../data/01_raw/Census_2010_Tracts"
OUTPUT_TABLES = "../output/final_paper/tables"
OUTPUT_FIGURES = "../output/final_paper/figures"

# Summary Statistics

## Crime Trends 

In [3]:
# Read unrestricted dataset into memory.
df = pd.read_csv(INPUT_DATA_PANEL)

# Reshape to long
triplet = get_value_variable_names(df, f"group_0_crimes_{constants.Analysis.MAIN_RESULTS_RADIUS}m")
weekly_value_vars_crime, month_to_int_dictionary, _ = triplet
df = pd.melt(df,
             id_vars=['case_number',
                      'judgment_for_plaintiff',
                      'latest_docket_month'],
             value_vars=weekly_value_vars_crime,
             var_name='month')
df.loc[:, 'month'] = df['month'].str[:7]  # Drop "_group_0_crimes_500m" from the end of each month

# Replace months with integers
df.loc[:, ['latest_docket_month', 'month']] = df[['latest_docket_month', 'month']].replace(month_to_int_dictionary)

# Calculate crime levels during each month relative to treatment, separately for treatment and control gropu                                    
df.loc[:, 'treatment_relative_month'] = df['month'] - df['latest_docket_month']
df = df.groupby(['judgment_for_plaintiff', 'treatment_relative_month'])['value'].agg(['mean'])
control_crime_trend = df.loc[(0, slice(constants.Analysis.MINIMUM_PRE_PERIOD,
                                       constants.Analysis.MAXIMUM_POST_PERIOD)),
                             :]
treatment_crime_trend = df.loc[(1, slice(constants.Analysis.MINIMUM_PRE_PERIOD,
                                         constants.Analysis.MAXIMUM_POST_PERIOD)),
                               :]    
fig, ax = plt.subplots()
ax.plot(control_crime_trend.index.get_level_values(1),
        control_crime_trend['mean'],
        color='grey',
        linestyle='--',
        label="Properties where Tenant Won Eviction Case")
ax.plot(treatment_crime_trend.index.get_level_values(1),
        treatment_crime_trend['mean'],
        color='black',
        linestyle='--',
        label="Properties where Plaintiff Won Eviction Case")
figure_utilities.plot_labeled_vline(ax, x=0, text="Latest\nDocket\nDate", color='black', linestyle='-',
                       text_y_location_normalized=0.1)
ax.set_xlabel("Month Relative to Latest Docket Date")
ax.set_ylabel(f"Crime Incidents within {constants.Analysis.MAIN_RESULTS_RADIUS} Meters")
ax.legend()

figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "crime_trends.png"))


## Map of Evictions, Colored by Poverty Rate in Census Tract

In [4]:
# Read unrestricted dataset into memory.
df = pd.read_csv(INPUT_DATA_PANEL)
df = df.reset_index() # So we can use 'case_number' like a column

# Create spatial data 
unrestricted_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']))
unrestricted_gdf = unrestricted_gdf.set_crs("EPSG:4326")
unrestricted_gdf = unrestricted_gdf.to_crs("EPSG:3857")

# Plot
fig, ax = plt.subplots(figsize=(8, 10))
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
unrestricted_gdf.plot(ax=ax,
                      color='black',
                      markersize=0.05)
cx.add_basemap(ax=ax, crs="EPSG:3857", source=cx.providers.CartoDB.Positron)

# Color census tracts by poverty rate.
boston_tracts_gdf = gpd.read_file(INPUT_DATA_BOSTON_TRACTS_SHAPEFILE)[['GEOID10', 'geometry']].set_index('GEOID10')
boston_tracts_gdf.index = boston_tracts_gdf.index.astype(int)
tract_poverty_rates_df = pd.read_csv(INPUT_DATA_TRACTS, usecols=['tract_geoid', 'poor_share2010'],
                                     index_col='tract_geoid')
boston_tracts_gdf = pd.concat([boston_tracts_gdf, tract_poverty_rates_df], axis=1).dropna(
    subset=['geometry', 'poor_share2010']).drop(index=25025990101)
boston_tracts_gdf.plot(ax=ax, column=boston_tracts_gdf['poor_share2010'], cmap='OrRd', alpha=0.4, legend=True,
                       legend_kwds={'label': "Poverty Rate of Census Tract",
                                    'orientation': "horizontal",
                                    'shrink': 0.25})

figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "evictions_map.png"))

## Binscatter of Crime and Eviction on Poverty

In [5]:
neighborhoods = pd.concat([(pd.read_csv(INPUT_DATA_NEIGHBORHOOD_CRIME_COUNTS, index_col='neighborhood')
                            .drop(index=['Bay Village', 'Leather District'])),
                           (df.groupby('neighborhood')['judgment_for_plaintiff']
                            .count()
                            .drop(index=['Bay Village'])
                            .rename('eviction_filings'))], axis=1)
neighborhood_populations = (pd.read_excel(INPUT_DATA_NEIGHBORHOODS, sheet_name="Age", usecols=[0, 1], header=5, skiprows=lambda row: row > 27)
                            .rename(columns={'Boston': 'neighborhood', 684379: 'population'})
                            .set_index('neighborhood'))
neighborhood_poverty_rates = (pd.read_excel(INPUT_DATA_NEIGHBORHOODS, sheet_name="Poverty Rates", usecols=[0, 3], header=5, skiprows=lambda row: row > 27)
                              .rename(columns={'Boston': 'neighborhood',  0.18914243501949649: 'poverty_rate'})
                              .set_index('neighborhood'))
neighborhoods = (neighborhoods
                 .merge(neighborhood_poverty_rates, how='left', left_index=True, right_index=True)
                 .merge(neighborhood_populations, how='left', left_index=True, right_index=True))

neighborhoods.loc[:, ['eviction_filings_per_resident', 'total_crimes_per_resident']] = (neighborhoods[['eviction_filings', 'total_crimes']]
                                                                                        .divide(neighborhoods['population'], axis=0)).values
neighborhoods.loc[:, ['eviction_filings_per_1000_residents']] = neighborhoods['eviction_filings_per_resident'] * 1000

fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True)
ax1.scatter(neighborhoods.dropna()['poverty_rate'],
           neighborhoods.dropna()['total_crimes_per_resident'],
           c='black')
ax1.set_ylabel("Crimes per Resident")
ax1.set_xlabel("Poverty Rate")
ax2.scatter(neighborhoods.dropna()['poverty_rate'],
           neighborhoods.dropna()['eviction_filings_per_1000_residents'],
           c='black')
ax2.set_ylabel("Eviction Filings per 1000 Residents")
ax2.set_xlabel("Poverty Rate")



def label_points(xs, ys, labels, ax):
    for x, y, label in zip(xs, ys, labels):
        if label in ["Roxbury", "South End", "West End", "Chinatown", "Mattapan", "Mission Hill", "Dorchester", "Jamaica Plain",
                     "Roslindale", "Back Bay"]:
            ax.text(x, y, label, horizontalalignment='center', fontsize='small')
label_points(neighborhoods.dropna().loc[:, 'poverty_rate'] - 0.04,
             neighborhoods.dropna().loc[:, 'total_crimes_per_resident'] + 0.005 ,
             neighborhoods.dropna().index,
             ax1)
label_points(neighborhoods.dropna().loc[:, 'poverty_rate'] + 0.04,
             neighborhoods.dropna().loc[:, 'eviction_filings_per_1000_residents'] + 0.1,
             neighborhoods.dropna().index,
             ax2)



for ax, variable, text_y_position in zip([ax1, ax2], ['total_crimes_per_resident', 'eviction_filings_per_1000_residents'], [0.35*(5.75/6), 5.75]):
    result = sm.WLS(endog=neighborhoods[variable],
                    exog=sm.add_constant(neighborhoods['poverty_rate']),
                    weights=neighborhoods['eviction_filings'],
                    missing='drop').fit()

    x_vals = [0.0, 0.4]
    y_vals = [result.params.iloc[0], result.params.iloc[0] + result.params.iloc[1] * 0.4]
    ax.plot(x_vals, y_vals, '-', c='black')

    text = "$\\frac{\\beta}{10}=" + str(round(result.params.iloc[1] / 10, 2)) + "$" + " (" + str(round(result.bse.iloc[1] / 10, 2)) + ")"
    ax.text(0.0, text_y_position, text)
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "evictions_binscatter.png"))

## Eviction Filings Over Time

In [6]:
# Plot the number of eviction filings over time.
df.loc[:, 'last_day_of_file_month'] = (pd.to_datetime(df['file_date']) +
                                       pd.tseries.offsets.MonthEnd(0))
filings_per_month = df.groupby('last_day_of_file_month')['case_number'].count()

# Plot eviction filing counts.
fig, ax = plt.subplots()
filings_per_month.plot(ax=ax, kind='line', color='black',
                       zorder=100)
ax.set_ylabel("Number of Evictions")
ax.set_xlabel("Month")
ax.grid(True)
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "filings_over_time.png"))


In [7]:
# Read unrestricted dataset into memory.
df = pd.read_csv(INPUT_DATA_PANEL)
df = df.reset_index() # So we can use 'case_number' like a column

# Produce summary statistics table.
treatment_date_variable = 'latest_docket_date'
outcomes_of_interest = [f'group_0_crimes_{constants.Analysis.MAIN_RESULTS_RADIUS}m']
summary_statistics_unrestricted, variable_display_names_dict = produce_summary_statistics(df)

# Rename columns.
summary_statistics_unrestricted.index = summary_statistics_unrestricted.index.set_names(["Panel", "Variable"])
column_display_names_dict = {'mean': "Mean", 'std': "S.D.", 'count': "N", '50%': 'Median'}

summary_statistics_unrestricted = summary_statistics_unrestricted.sort_values(['Panel', 'Variable'])
# Keep only outcomes of interest
outcomes = constants.Variables.outcomes.copy()
for outcome in outcomes:
    if outcome not in outcomes_of_interest:
        if f"relative_pre_treatment_change_in_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"relative_pre_treatment_change_in_{outcome}",
                                                                                   level=1, axis=0)
        if f"pre_treatment_change_in_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"pre_treatment_change_in_{outcome}",
                                                                                   level=1, axis=0)
        if f"total_twenty_seventeen_{outcome}" in summary_statistics_unrestricted.index.get_level_values(1):
            summary_statistics_unrestricted = summary_statistics_unrestricted.drop(f"total_twenty_seventeen_{outcome}",
                                                                                   level=1, axis=0)

# Drop median column.
summary_statistics_unrestricted = summary_statistics_unrestricted.drop(columns='50%')

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "summary_statistics.tex")
latex = (summary_statistics_unrestricted
         .rename(index=variable_display_names_dict)
         .rename(columns=column_display_names_dict)
         .style
         .format(formatter={
    'Mean': "{:,.2f}",
    'Median': "{:,.2f}",
    'S.D.': "{:,.2f}",
    'N': "{:,.0f}"})
         .format_index("\\textit{{{}}}", escape="latex", axis=0, level=0)
         .to_latex(None,
                   column_format="llcccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{3cm}")
with open(filename, 'w') as file:
    file.write(latex)
summary_statistics_unrestricted

mean  \
Panel                                 Variable                                                           
Panel A: Pre-treatment Outcomes       pre_treatment_change_in_group_0_crimes_300m           -63.256957   
                                      relative_pre_treatment_change_in_group_0_crimes...     -6.120782   
                                      total_twenty_seventeen_group_0_crimes_300m            470.282416   
Panel B: Census Tract Characteristics frac_coll_plus2010                                      0.315315   
                                      job_density_2013                                    16714.101631   
                                      med_hhinc2016                                       46250.259325   
                                      poor_share2010                                          0.288777   
                                      popdensity2010                                      23449.588205   
                                      share_white2010                                         0.314013   
Panel C: Case Initiation              for_cause                                               0.114269   
                                      isEntityD                                               0.023683   
                                      non_payment                                             0.762581   
Panel D: Case Resolution              case_duration                                          20.096552   
                                      defaulted                                               0.464180   
                                      dismissed                                               0.433985   
                                      judgment                                             1488.317940   

                                                                                                   std  \
Panel                                 Variable                                                           
Panel A: Pre-treatment Outcomes       pre_treatment_change_in_group_0_crimes_300m           135.221051   
                                      relative_pre_treatment_change_in_group_0_crimes...     29.007428   
                                      total_twenty_seventeen_group_0_crimes_300m            326.841123   
Panel B: Census Tract Characteristics frac_coll_plus2010                                      0.216676   
                                      job_density_2013                                    43368.358735   
                                      med_hhinc2016                                       24935.086199   
                                      poor_share2010                                          0.152398   
                                      popdensity2010                                      14401.768551   
                                      share_white2010                                         0.268460   
Panel C: Case Initiation              for_cause                                               0.318232   
                                      isEntityD                                               0.152104   
                                      non_payment                                             0.425627   
Panel D: Case Resolution              case_duration                                          25.060743   
                                      defaulted                                               0.498863   
                                      dismissed                                               0.495770   
                                      judgment                                             3429.810758   

                                                                                           count  
Panel                                 Variable                                                    
Panel A: Pre-treatment Outcomes       pre_treatment_change_in_group_0_crimes_300m 

In [8]:
# Produce treatment timings table.
treatment_timings = (df
                     .groupby(['latest_docket_month', 'judgment_for_plaintiff'])['case_number']
                     .count()
                     .reset_index()
                     .fillna(0))
treatment_timings = treatment_timings.pivot(index='latest_docket_month', columns='judgment_for_plaintiff').fillna(0)
treatment_timings.columns = ["Cases Won By Defendant", "Cases Won By Plaintiff"]
portion_of_all_cases = (treatment_timings['Cases Won By Plaintiff'] + treatment_timings[
    'Cases Won By Defendant']) / len(df)
treatment_timings = pd.concat([treatment_timings, portion_of_all_cases.rename('Portion of All Cases')], axis=1)
sum_across_filing_date = pd.DataFrame(treatment_timings.sum(axis=0)).T
sum_across_filing_date.index = ["All Months"]
treatment_timings = pd.concat([sum_across_filing_date, treatment_timings], axis=0)
treatment_timings.index = treatment_timings.index.rename("Last Docket Date")

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "treatment_timings.tex")
treatment_timings.style.format(formatter={'Cases Won By Plaintiff': '{:,.0f}',
                                          'Cases Won By Defendant': '{:,.0f}',
                                          'Portion of All Cases': '{:0.2f}'}).to_latex(filename, column_format="lccc",
                                                                                       hrules=True)
treatment_timings

,Cases Won By Defendant,Cases Won By Plaintiff,Portion of All Cases
Last Docket Date,,,
All Months,734.0,955.0,1.000000
2019-04,1.0,5.0,0.003552
2019-05,1.0,7.0,0.004737
2019-06,51.0,24.0,0.044405
2019-07,64.0,47.0,0.065719
2019-08,68.0,110.0,0.105388
2019-09,76.0,101.0,0.104796
2019-10,90.0,98.0,0.111308
2019-11,62.0,76.0,0.081705


In [9]:
# Calculate percent of cases which are in high poverty neighborhoods.
df = df.loc[df['judgment_for_plaintiff'] == 1, :]
original_N = len(df)
cases_in_poor_tracts = len(df.loc[df['poor_share2010'] > 0.20, :])
cases_in_poor_tracts / original_N

0.6408376963350786